In [13]:
import sys
!{sys.executable} -m pip install geopy dash dash-bootstrap-components

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# origem dos dados

corridas_df = pd.read_csv("ride_v2.csv", sep=";", dtype=str)
enderecos_df = pd.read_csv("rideaddress_v1.csv", sep=";", dtype=str)
estimativas_df = pd.read_csv("rideestimative_v3.csv", sep=";", dtype=str)
produtos_df = pd.read_csv("product.csv", sep=";", dtype=str)

# padronização dos dados

corridas_df["Schedule"] = pd.to_datetime(corridas_df["Schedule"], errors="coerce")
for df in [corridas_df, enderecos_df, estimativas_df]:
    df["RideID"] = df["RideID"].astype(str).str.replace(".0", "", regex=False)

# colunas derivadas

corridas_df["DiaSemana"] = corridas_df["Schedule"].dt.weekday
corridas_df["HoraDia"] = corridas_df["Schedule"].dt.hour
corridas_df["Minuto"] = corridas_df["Schedule"].dt.minute
corridas_df["HoraDecimal"] = corridas_df["HoraDia"] + corridas_df["Minuto"] / 60
corridas_df["Janela15"] = corridas_df["Schedule"].dt.floor("15min")
tempo_df = corridas_df[["RideID", "DiaSemana", "HoraDia", "Minuto", "HoraDecimal", "Janela15"]].dropna()

# usa as coord, de origem e destinos

enderecos_df = enderecos_df.rename(columns={"RideAddressTypeID": "TipoEndereco"})
origem_df = enderecos_df[enderecos_df["TipoEndereco"] == "1"][["RideID", "Lat", "Lng", "Address"]].rename(columns={"Lat": "LatOrig", "Lng": "LngOrig", "Address": "EnderecoOrig"})
destino_df = enderecos_df[enderecos_df["TipoEndereco"] == "2"][["RideID", "Lat", "Lng", "Address"]].rename(columns={"Lat": "LatDest", "Lng": "LngDest", "Address": "EnderecoDest"})
coordenadas_df = pd.merge(origem_df, destino_df, on="RideID", how="inner")

for col in ["LatOrig", "LngOrig", "LatDest", "LngDest"]:
    coordenadas_df[col] = coordenadas_df[col].str.replace(",", ".").astype(float).round(6)

#junta estimativa e produtoss

estimativas_df["ProductID"] = estimativas_df["ProductID"].astype(str)
produtos_df["ProductID"] = produtos_df["ProductID"].astype(str)
estimativas_merge = pd.merge(estimativas_df, produtos_df, on="ProductID", how="left")
estimativas_merge["Price"] = estimativas_merge["Price"].str.replace(",", ".").astype(float)

dicionario_estimativas = estimativas_merge.groupby("RideID").apply(
    lambda x: dict(zip(x["Description"], x["Price"]))
).reset_index().rename(columns={0: "Estimativas"})

# filtragem de IDs

coordenadas_df["RideID"] = coordenadas_df["RideID"].astype(str)
dicionario_estimativas["RideID"] = dicionario_estimativas["RideID"].astype(str)
ride_ids_validos = set(tempo_df["RideID"]) & set(coordenadas_df["RideID"]) & set(dicionario_estimativas["RideID"])

tempo_df = tempo_df[tempo_df["RideID"].isin(ride_ids_validos)].sort_values("RideID").reset_index(drop=True)
coordenadas_df = coordenadas_df[coordenadas_df["RideID"].isin(ride_ids_validos)].sort_values("RideID").reset_index(drop=True)
dicionario_estimativas = dicionario_estimativas[dicionario_estimativas["RideID"].isin(ride_ids_validos)].sort_values("RideID").reset_index(drop=True)

# concatenação dos dataframes
base_modelo_df = pd.concat([
    tempo_df,
    coordenadas_df.drop(columns=["RideID"]),
    dicionario_estimativas.drop(columns=["RideID"])
], axis=1)

# calculo da distancia
base_modelo_df = base_modelo_df.dropna(subset=["LatOrig", "LngOrig", "LatDest", "LngDest"]).reset_index(drop=True)
base_modelo_df["DistanciaKM"] = base_modelo_df.apply(
    lambda row: geodesic((row["LatOrig"], row["LngOrig"]), (row["LatDest"], row["LngDest"])).kilometers,
    axis=1
)

print("\nBase pronta para modelagem:")
print(base_modelo_df.head())

# o que vai modelar ()
servicos_para_modelar = ["UberX", "99POP", "Taxi"]

# modelos de teste
resultados_finais = []

for servico in servicos_para_modelar:
    print(f"\Modelo para: {servico}")
    dados_servico = base_modelo_df[base_modelo_df["Estimativas"].apply(
        lambda est: isinstance(est, dict) and servico in est and isinstance(est[servico], (int, float))
    )].copy()

    dados_servico["PrecoAlvo"] = dados_servico["Estimativas"].apply(lambda est: est[servico])

    q1 = dados_servico["PrecoAlvo"].quantile(0.25) ###### Remoção de outliers
    q3 = dados_servico["PrecoAlvo"].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr
    dados_servico = dados_servico[(dados_servico["PrecoAlvo"] >= limite_inferior) & (dados_servico["PrecoAlvo"] <= limite_superior)]

    colunas_basicas = ["DistanciaKM", "DiaSemana", "HoraDia", "HoraDecimal", "LatOrig", "LngOrig", "LatDest", "LngDest"]

    outros_servicos = set()
    dados_servico["Estimativas"].apply(lambda est: outros_servicos.update(est.keys()) if isinstance(est, dict) else None)
    outros_servicos.discard(servico)

    for s in outros_servicos:
        nome_col = f"preco_{s.lower().replace(' ', '_')}"
        dados_servico[nome_col] = dados_servico["Estimativas"].apply(lambda est: est.get(s) if isinstance(est, dict) else np.nan)

    # contrução do modelo

    colunas_entrada = colunas_basicas + [col for col in dados_servico.columns if col.startswith("preco_")]
    X = dados_servico[colunas_entrada].fillna(-1)
    y = dados_servico["PrecoAlvo"]

    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
    arvore_modelo = DecisionTreeRegressor(random_state=42)
    arvore_modelo.fit(X_treino, y_treino)
    y_predito = arvore_modelo.predict(X_teste)

    # avaliação
    mae = mean_absolute_error(y_teste, y_predito)
    rmse = np.sqrt(mean_squared_error(y_teste, y_predito))
    r2 = r2_score(y_teste, y_predito)

    resultados_finais.append({
        "Servico": servico,
        "Registros": len(X),
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R2": round(r2, 4)
    })

    print(f"Modelo para {servico} avaliado com sucesso!")
    print(f" → MAE : R${mae:.2f}")
    print(f" → RMSE: R${rmse:.2f}")
    print(f" → R²  : {r2:.4f}")

#resumo

resumo_df = pd.DataFrame(resultados_finais).sort_values(by="R2", ascending=False)
print("\nModelos Finais:")
print(resumo_df.to_string(index=False))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 3.9 MB/s eta 0:00:00


<ipython-input-13-2676a076380c>:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dicionario_estimativas = estimativas_merge.groupby("RideID").apply(



Base pronta para modelagem:
    RideID  DiaSemana  HoraDia  Minuto  HoraDecimal            Janela15  \
0  1183200          1       10       9    10.150000 2021-08-17 10:00:00   
1  1183201          1       10       9    10.150000 2021-08-17 10:00:00   
2  1183202          1       10      10    10.166667 2021-08-17 10:00:00   
3  1183203          1       10      10    10.166667 2021-08-17 10:00:00   
4  1183204          1       10      10    10.166667 2021-08-17 10:00:00   

     LatOrig    LngOrig                                       EnderecoOrig  \
0 -26.329754 -48.840428  Rua João Pinheiro, 585 - Rua João Pinheiro - B...   
1 -27.491979 -48.528288  Rodovia Rafael da Rocha Pires, 1883 - Rodovia ...   
2 -19.849580 -44.019916  Rua Barão do Rio Branco, 12 - Rua Barão do Rio...   
3 -23.962423 -46.254658               Tv. Duzentos e Sessenta e Um, 72, 72   
4 -10.919802 -37.077442        Rua Argentina, 160 - Rua Argentina - Brasil   

     LatDest    LngDest                            

In [15]:
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc

# app Dash
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H2("Dashboard - Análise de Preços por Serviço"),
    dcc.Dropdown(
        id="dropdown-servico",
        options=[{"label": s, "value": s} for s in ["UberX", "99POP", "Taxi"]],
        value="UberX",
        style={"width": "300px"}
    ),
    html.Div([
        dcc.Graph(id="grafico-dia"),
        dcc.Graph(id="grafico-distancia"),
        dcc.Graph(id="grafico-previsao")
    ])
])

@app.callback(
    [Output("grafico-dia", "figure"),
     Output("grafico-distancia", "figure"),
     Output("grafico-previsao", "figure")],
    [Input("dropdown-servico", "value")]
)
def atualizar_graficos(servico):
    df = base_modelo_df[base_modelo_df["Estimativas"].apply(lambda est: isinstance(est, dict) and servico in est)].copy()
    df["Preco"] = df["Estimativas"].apply(lambda est: est[servico])

    # 1. Preço médio por dia da semana
    fig1 = px.bar(
        df.groupby("DiaSemana")["Preco"].mean().reset_index(),
        x="DiaSemana", y="Preco", title="Preço médio por dia da semana"
    )

    # 2. Preço vs Distância
    fig2 = px.scatter(
        df, x="DistanciaKM", y="Preco", title="Preço vs Distância",
        trendline="ols", opacity=0.6
    )

    # 3. Preço real vs preço previsto (aqui usamos HoraDecimal como base para o eixo X)
    fig3 = px.scatter(
        df, x="HoraDecimal", y="Preco", title="Preço real vs Hora do Dia",
        opacity=0.6
    )

    return fig1, fig2, fig3

# Executar o servidor Dash
if __name__ == '__main__':
    app.run(debug=False)

<IPython.core.display.Javascript object>